<a href="https://colab.research.google.com/github/G-Gopan/OGPT/blob/main/OpenGPT_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# **!!! Switch the runtime to GPU !!!**

In [ ]:
# Install OpenGPT
! pip install --upgrade opengpt
! pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20

In [ ]:
#import Libraries
import pandas as pd
import datasets
import nltk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from opengpt.config import Config
from opengpt.model_utils import add_tokens_to_model_and_tokenizer
from opengpt.dataset_utils import create_labels, pack_examples
from opengpt.data_collator import DataCollatorWithPadding

In [ ]:
# Download the configs and data from OpenGPT repo https://github.com/G-Gopan/OGPT
!wget https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/data/nhs_uk_full/prepared_generated_data_for_nhs_uk_qa.csv
!wget https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/data/nhs_uk_full/prepared_generated_data_for_nhs_uk_conversations.csv
!wget https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/data/medical_tasks_gpt4/prepared_generated_data_for_medical_tasks.csv
!wget https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/configs/example_train_config.yaml
!wget https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/data/Leukaemia.csv

--2024-05-20 12:03:22--  https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/data/nhs_uk_full/prepared_generated_data_for_nhs_uk_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10370226 (9.9M) [text/plain]
Saving to: ‘prepared_generated_data_for_nhs_uk_qa.csv’

prepared_generated_ 100%[===================>]   9.89M  --.-KB/s    in 0.1s    

2024-05-20 12:03:22 (92.4 MB/s) - ‘prepared_generated_data_for_nhs_uk_qa.csv’ saved [10370226/10370226]

--2024-05-20 12:03:22--  https://raw.githubusercontent.com/G-Gopan/OGPT/main/OpenGPT-main/data/nhs_uk_full/prepared_generated_data_for_nhs_uk_conversations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.

In [ ]:
# Load the config
config = Config(yaml_path='./example_train_config.yaml')
config.train.to_dict()

{'model': 'olm/olm-gpt2-oct-2022',
 'datasets': ['../data/example_project_data/prepared_generated_data_for_example_project.csv',
  '../data/nhs_uk_full/prepared_generated_data_for_nhs_uk_qa.csv',
  '../data/nhs_uk_full/prepared_generated_data_for_nhs_uk_conversations.csv',
  '../data/medical_tasks_gpt4/prepared_generated_data_for_medical_tasks.csv'],
 'ignore_index': -100,
 'max_seq_len': 512,
 'packing_type': 'partial',
 'shuffle_dataset': True,
 'hf_training_arguments': {'output_dir': '../data/results/',
  'gradient_accumulation_steps': 16,
  'per_device_eval_batch_size': 1,
  'per_device_train_batch_size': 1,
  'load_best_model_at_end': False,
  'learning_rate': 2e-05,
  'weight_decay': 0.1,
  'adam_beta1': 0.9,
  'adam_beta2': 0.95,
  'adam_epsilon': 1e-07,
  'max_grad_norm': 1,
  'num_train_epochs': 1,
  'lr_scheduler_type': 'cosine',
  'warmup_ratio': 0.03,
  'logging_strategy': 'steps',
  'logging_steps': 100,
  'save_strategy': 'steps',
  'save_steps': 30000,
  'seed': 11,
  'o

In [ ]:
# Load the GPT-2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained(config.train.model)
tokenizer = AutoTokenizer.from_pretrained(config.train.model)
tokenizer.model_max_length = config.train.max_seq_len

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/805k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/463k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# Expand tokenizer/model with the necessary special tokens for conversational LLMs
add_tokens_to_model_and_tokenizer(config, tokenizer, model)

In [ ]:
# Define the datasets to be used (a list of CSV paths). The datasets used here are high quality medical datasets
config.train.datasets = ['./prepared_generated_data_for_nhs_uk_qa.csv',
                         './prepared_generated_data_for_nhs_uk_conversations.csv',
                         './prepared_generated_data_for_medical_tasks.csv']

In [ ]:
# Load datasets and shuffle if needed
train_dataset = datasets.Dataset.from_csv(config.train.datasets)
if config.train.shuffle_dataset:
    train_dataset = train_dataset.shuffle()
    print("Shuffling dataset!")

Generating train split: 0 examples [00:00, ? examples/s]

Shuffling dataset!


In [ ]:
# Check the size of the dataset and columns
train_dataset

Dataset({
    features: ['text', 'raw_data_id'],
    num_rows: 31708
})

In [ ]:
# the text part will be used to train our model, note the special tokens that are required for training
# models with OpenGPT (check the config file in the OpenGPT repo for explanations)
train_dataset[0]['text']

"<|user|> Explain the process of diagnosing a patient with chronic obstructive pulmonary disease (COPD). <|eos|> <|ai|> Diagnosing a patient with chronic obstructive pulmonary disease (COPD) involves a series of steps: \n1. Taking a thorough medical history to understand the patient's symptoms, risk factors, and family history. \n2. Performing a physical examination, focusing on the respiratory system and checking for signs such as wheezing or decreased breath sounds.\n3. Conducting a pulmonary function test (PFT), particularly spirometry, which measures the volume and speed of air that the patient can inhale and exhale. A reduced forced expiratory volume in 1 second (FEV1) and a low FEV1/FVC (forced vital capacity) ratio can indicate COPD. \n4. Ordering a chest X-ray or CT scan to look for physical abnormalities in the lungs, such as hyperinflation or bullae.\n5. Performing a complete blood count (CBC) and arterial blood gas (ABG) test to assess oxygen and carbon dioxide levels in the

In [ ]:
# Remove everything but text
to_remove = list(train_dataset.column_names)
to_remove.remove('text')
train_dataset = train_dataset.remove_columns(to_remove)

In [ ]:
# Ignore max_seq_len warning, it is handled by the packer or data_collator
train_dataset = train_dataset.map(
    lambda examples: tokenizer(examples['text'], add_special_tokens=False),
    batched=True,
    num_proc=1,
    remove_columns=["text"])
# Create labels for supervised training (meaning we do not train on questions, but only on answers)
train_dataset = train_dataset.map(
    lambda examples: create_labels(examples, config, tokenizer),
    batched=True,
    batch_size=1000,
    num_proc=1,
)
# We only do packing for the train set
train_dataset = train_dataset.map(
    lambda examples: pack_examples(examples, config.train.max_seq_len, packing_type=config.train.packing_type),
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Map:   0%|          | 0/31708 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (660 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/31708 [00:00<?, ? examples/s]

Map:   0%|          | 0/31708 [00:00<?, ? examples/s]

In [ ]:
# Check the new train_dataset (take note of how the labels look). The USER (Question) part of the input should have a label of -100,
# and the AI part (Answer) should have labels equal to input_ids
for i in range(10):
    print(train_dataset[0]['input_ids'][i], train_dataset[0]['labels'][i], train_dataset[0]['attention_mask'][i])

50265 -100 1
48142 -100 1
225 -100 1
88 -100 1
76 -100 1
73 -100 1
1149 -100 1
292 -100 1
49879 -100 1
225 -100 1


In [ ]:
# Load the HF training arguments and make the data collator
training_args = TrainingArguments(**config.train.hf_training_arguments.to_dict())
dc = DataCollatorWithPadding(tokenizer.pad_token_id, config.train.ignore_index, max_seq_len=config.train.max_seq_len)

# We are using HF trainer to train our models
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=None,
    data_collator=dc,
)

In [ ]:
# Run training, should take around 30min.
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,1.604500
200,1.442000
300,1.399000
400,1.375900
500,1.347800
600,1.350500
700,1.340000


TrainOutput(global_step=735, training_loss=1.4040042124637941, metrics={'train_runtime': 1624.3099, 'train_samples_per_second': 7.241, 'train_steps_per_second': 0.452, 'total_flos': 2506604062464000.0, 'train_loss': 1.4040042124637941, 'epoch': 0.999829960891005})

# Testing

In [ ]:
gen = pipeline(model=model, tokenizer=tokenizer, task='text-generation', device=model.device)

In [ ]:
#Let's ask the trained model some questions
t = "<|user|> What is leukaemia? <|eos|> <|ai|>" # The format with special tokens is required, because of training
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

# Save DataFrame to a CSV file
df.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<|user|> What is leukaemia? <|eos|> <|ai|> Leukaemia is a condition where the body's cells become too weak to produce enough oxygen. It's a common cause of cancer in the UK.
References:
- https://www.nhs.uk/conditions/cancer/causes/  <|eod|> <|eod|> <|ai|> The patient's symptoms include a high temperature, shortness of breath, and a high temperature. The doctor may recommend a blood test to check for any abnormalities in the blood.
References:
- https://
User questions and LLM responses saved to user_llm_data.csv


In [ ]:
t = "<|user|> What are the symptoms of leukaemia? <|eos|> <|ai|>"
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read user_llm_data.csv
user_llm = pd.read_csv('user_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_user_llm = pd.concat([user_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_user_llm.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

<|user|> What are the symptoms of leukaemia? <|eos|> <|ai|> The symptoms of leukaemia include feeling tired, feeling sick, and feeling sick.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> Symptoms of leukaemia include feeling tired, feeling sick, and feeling sick.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> The symptoms of leukaemia can include a high temperature, feeling hot, and feeling sick.
References:

User questions and LLM responses saved to user_llm_data.csv


In [ ]:
t = "<|user|> Can leukaemia be cured? <|eos|> <|ai|>"
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read user_llm_data.csv
user_llm = pd.read_csv('user_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_user_llm = pd.concat([user_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_user_llm.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

<|user|> Can leukaemia be cured? <|eos|> <|ai|> Yes, treatment for leukaemia can be effective in some cases. However, it is important to discuss the treatment options with your doctor.
References:
- https://www.nhs.uk/conditions/leukaemia/treatment/  <|eod|> <|ai|> Treatment for leukaemia involves a combination of lifestyle changes, medication, and surgery. The main treatment options include surgery, where a small incision is made in the neck to remove the cancerous cells, and chemotherapy, where a small amount
User questions and LLM responses saved to user_llm_data.csv


In [ ]:
t = "<|user|> Is leukaemia a type of cancer? <|eos|> <|ai|>"
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read user_llm_data.csv
user_llm = pd.read_csv('user_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_user_llm = pd.concat([user_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_user_llm.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

<|user|> Is leukaemia a type of cancer? <|eos|> <|ai|> Leukaemia is a type of cancer that can affect the immune system, causing symptoms such as a high temperature, feeling sick, and feeling sick.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|>: The symptoms of a rare but serious condition called aortic aneurysm can include a sudden onset of chest pain, a high temperature, and a high temperature.
References:
- https://www.nhs.uk/
User questions and LLM responses saved to user_llm_data.csv


In [ ]:
# And a couple more quesetions, e.g. an MCQ (The correct answer is A)
t = "<|user|> Choose the correct option about leukaemia. a) leukaemia is a type of blood cancer. b) leukaemia is a kind of fruit. <|eos|> <|ai|>"
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read user_llm_data.csv
user_llm = pd.read_csv('user_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_user_llm = pd.concat([user_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_user_llm.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

<|user|> Choose the correct option about leukaemia. a) leukaemia is a type of blood cancer. b) leukaemia is a kind of fruit. <|eos|> <|ai|> Choose the correct option about leukaemia. a) leukaemia is a type of blood cancer. b) leukaemia is a type of fruit.  <|eod|> <|ai|> Leukaemia is a type of blood cancer that can affect the body's ability to produce enough oxygen to survive. It is a serious condition that requires lifelong treatment.  
User questions and LLM responses saved to user_llm_data.csv


In [ ]:
t = "<|user|> I had a high fever for the past 3 days, do i have leukaemia? <|eos|> <|ai|>"
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read user_llm_data.csv
user_llm = pd.read_csv('user_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_user_llm = pd.concat([user_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_user_llm.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

<|user|> I had a high fever for the past 3 days, do i have leukaemia? <|eos|> <|ai|> Yes, you should see a GP if you have a high fever.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> The symptoms of high fever can include a high temperature, chills, a high temperature, and a high temperature. It's important to get medical attention if you experience any of these symptoms.
References:
- https://www.nhs.uk/conditions/high
User questions and LLM responses saved to user_llm_data.csv


In [ ]:
t = "<|user|> Tell me something about leukaemia. <|eos|> <|ai|>"
response = gen(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read user_llm_data.csv
user_llm = pd.read_csv('user_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_user_llm = pd.concat([user_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_user_llm.to_csv('user_llm_data.csv', index=False)
print("User questions and LLM responses saved to user_llm_data.csv")

<|user|> Tell me something about leukaemia. <|eos|> <|ai|> Leukaemia is a type of cancer that affects the blood vessels in the brain. It is a type of cancer that affects the blood cells that supply the brain with oxygen. It is a serious condition that requires prompt medical attention.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> The symptoms of leukaemia can include a high temperature, feeling sick, feeling tired, feeling sick, feeling sick, feeling sick
User questions and LLM responses saved to user_llm_data.csv


In [ ]:
#Read training dataset.csv
training = pd.read_csv('/content/Leukaemia.csv')

# Convert the text data in both DataFrames to lowercase
training ['text'] = training ['text'].str.lower()
new_user_llm['text'] = new_user_llm['text'].str.lower()

# Create TF-IDF vectors for the training data and response data
vectorizer = TfidfVectorizer()
training_tfidf = vectorizer.fit_transform(training ['text'])
response_tfidf = vectorizer.transform(new_user_llm['text'])

# Compute cosine similarity between response data and training data
similarity_scores = cosine_similarity(response_tfidf, training_tfidf)
average_similarity_score = np.mean(similarity_scores)

# Print the average similarity score
print(f"Average Cosine Similarity Score:{average_similarity_score * 100:.2f}%")

Average Cosine Similarity Score:12.42%


In [ ]:
# List of test prompts and their references
test_prompts = [
    ("<|user|> What is leukaemia? <|eos|> <|ai|>", "Leukaemia is a type of blood cancer."),
    ("<|user|> What are the symptoms of leukaemia? <|eos|> <|ai|>", "Symptoms of leukaemia include fatigue, fever, frequent infections, and bleeding or bruising easily."),
    ("<|user|> Can leukaemia be cured? <|eos|> <|ai|>", "Treatment can lead to remission, but a cure is not always possible."),
    ("<|user|> Is leukaemia a type of cancer? <|eos|> <|ai|>", "Yes, leukaemia is a type of blood cancer."),
    ("<|user|> Choose the correct option about leukaemia. a) leukaemia is a type of blood cancer. b) leukaemia is a kind of fruit. <|eos|> <|ai|>", "a) leukaemia is a type of blood cancer."),
    ("<|user|> I had a high fever for the past 3 days, do I have leukaemia? <|eos|> <|ai|>", "High fever can be a symptom, but it does not necessarily mean you have leukaemia. Consult a doctor for proper diagnosis."),
    ("<|user|> Tell me something about leukaemia. <|eos|> <|ai|>", "Leukaemia is a type of cancer that affects the blood and bone marrow.")
]

references = [ref for a, ref in test_prompts]
predictions = []

# Generate predictions
for prompt, a in test_prompts:
    response = gen(prompt, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
    predictions.append(response)
    print(response)

<|user|> What is leukaemia? <|eos|> <|ai|> Leukaemia is a type of cancer that can affect the blood and bone marrow. It is caused by a faulty gene that is passed on from mother to child.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> Leukaemia is a rare but serious cancer that can affect the blood and bone marrow. It is caused by a faulty gene that is passed on from mother to child.
References:
- https://www.nhs.uk
<|user|> What are the symptoms of leukaemia? <|eos|> <|ai|> The symptoms of leukaemia include a high temperature, feeling sick, feeling tired, and being sick.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> Symptoms of leukaemia include a high temperature, feeling sick, feeling tired, and being sick.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> The symptoms of leukaemia can include a high temperature, feeling


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<|user|> Can leukaemia be cured? <|eos|> <|ai|> Yes, leukaemia can be cured. However, it is important to monitor the condition closely and to seek medical advice if you're worried about your chances of developing the condition.
References:
- https://www.nhs.uk/conditions/leukaemia/  <|eod|> <|ai|> The most common cause of leukaemia is a faulty gene that causes the cells to produce too much oxygen. This can lead to a range of symptoms, including fatigue, weight loss, and a high temperature
<|user|> Is leukaemia a type of cancer? <|eos|> <|ai|> Yes, leukaemia is a type of cancer that can be caused by a variety of conditions, including a genetic mutation, a chromosomal abnormality, or a genetic disorder.
References:
- https://www.nhs.uk/conditions/leukaemia/causes/  <|eod|> <|ai|> Leukaemia is a type of cancer that can be caused by a variety of conditions, including a faulty gene, a faulty gene mutation, a faulty gene mutation, a faulty
<|user|> Choose the correct option about leukaemia. 

In [ ]:
# Tokenize references and predictions
tokenized_references = [tokenizer.encode(ref, add_special_tokens=False) for ref in references]
tokenized_predictions = [tokenizer.encode(pred, add_special_tokens=False) for pred in predictions]

# Ensure that each list of tokens is the same length by truncating or padding
max_len = max(len(seq) for seq in tokenized_references + tokenized_predictions)
tokenized_references = [seq + [tokenizer.pad_token_id] * (max_len - len(seq)) for seq in tokenized_references]
tokenized_predictions = [seq + [tokenizer.pad_token_id] * (max_len - len(seq)) for seq in tokenized_predictions]

# Flatten the lists
flat_references = [token for seq in tokenized_references for token in seq]
flat_predictions = [token for seq in tokenized_predictions for token in seq]

# Calculate Precision, Recall, and F1 Score
precision = precision_score(flat_references, flat_predictions, average='macro', zero_division=1)
recall = recall_score(flat_references, flat_predictions, average='macro', zero_division=1)
f1 = f1_score(flat_references, flat_predictions, average='macro', zero_division=1)

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")

Precision: 12.65%
Recall: 70.33%
F1 Score: 0.14%


In [ ]:
# Calculate BLEU scores
bleu_scores = []
for pred, ref in zip(predictions, references):
    score = sentence_bleu([ref.split()], pred.split())
    bleu_scores.append(score)

# Print average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score: {average_bleu_score * 100:.2f}%")

Average BLEU score: 3.40%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


# **Untrained Model for Comparision**

In [ ]:
# Finally, to show that our training works, we will also try to query an untrained model
model_not_trained = AutoModelForCausalLM.from_pretrained(config.train.model)
tokenizer_not_trained = AutoTokenizer.from_pretrained(config.train.model)
gen_not_trained = pipeline(model=model_not_trained, tokenizer=tokenizer_not_trained, task='text-generation', device=model.device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
t = "What is leukaemia" # No special tokens, as this model is not trained.
response = gen_not_trained(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

# Save DataFrame to a CSV file
df.to_csv('untrained_llm_data.csv', index=False)
print("User questions and LLM responses saved to untrained_llm_data.csv")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


What is leukaemia?
Leukaemia is a type of cancer that is caused by the cells in the body that are unable to produce enough blood to survive. It is a type of cancer that can be caused by a variety of factors, including a variety of different types of cancer.
What is the difference between leukaemia and lymphoma?
Leukaemia is a type of cancer that is caused by the cells in the body that are unable to produce enough blood to survive. It is a type of cancer that
User questions and LLM responses saved to untrained_llm_data.csv


In [ ]:
t = "What are the symptoms of leukaemia?" # No special tokens, as this model is not trained.
response = gen_not_trained(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read untrained_llm_data.csv
untrained_llm = pd.read_csv('untrained_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_untrained_llm = pd.concat([untrained_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_untrained_llm.to_csv('untrained_llm_data.csv', index=False)
print("User questions and LLM responses saved to untrained_llm_data.csv")


What are the symptoms of leukaemia?
Leukaemia is a disease of the blood cells that causes the body to produce more blood. This is because the body’s cells are unable to produce enough blood to keep the body healthy.
Symptoms of leukaemia include:
– Redness in the skin
– Redness in the eyes
– Redness in the throat
– Redness in the chest
– Redness in the abdomen
– Redness in
User questions and LLM responses saved to untrained_llm_data.csv


In [ ]:
t = "What is the cure for leukaemia?" # Let's try a general question
response = gen_not_trained(t, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
print(response)

#Read untrained_llm_data.csv
untrained_llm = pd.read_csv('untrained_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_untrained_llm = pd.concat([untrained_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_untrained_llm.to_csv('untrained_llm_data.csv', index=False)
print("User questions and LLM responses saved to untrained_llm_data.csv")

What is the cure for leukaemia?
The cure for leukaemia is the treatment of the disease. The treatment of leukaemia is the same as for any other cancer. The treatment of leukaemia is the same as for any other cancer. The treatment of leukaemia is the same as for any other cancer.
What is the cure for cancer?
The cure for cancer is the treatment of the disease. The cure for cancer is the same as for any
User questions and LLM responses saved to untrained_llm_data.csv


In [ ]:
t = "Tell me something about leukaemia."
response = gen_not_trained(t, do_sample=True, max_length=256, temperature=0.2)[0]['generated_text']
print(response)

#Read untrained_llm_data.csv
untrained_llm = pd.read_csv('untrained_llm_data.csv')

# Store user questions and LLM responses to a DataFrame
data = {'text': [response],'raw_data_id': [0]}
df = pd.DataFrame(data)

new_untrained_llm = pd.concat([untrained_llm, df], ignore_index=True)

# Save DataFrame to a CSV file
new_untrained_llm.to_csv('untrained_llm_data.csv', index=False)
print("User questions and LLM responses saved to untrained_llm_data.csv")

Tell me something about leukaemia.
I’m not sure if it’s a disease, but it’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States.
I’m not sure if it’s a disease, but it’s a disease that’s very common in the United States. It’s a disease that’s very common in the United States. It’s a disease that’s
User questions and LLM responses saved to untrained_llm_data.csv


In [ ]:
#Read training dataset.csv
training = pd.read_csv('/content/Leukaemia.csv')

# Convert the text data in both DataFrames to lowercase
training ['text'] = training ['text'].str.lower()
new_untrained_llm['text'] = new_untrained_llm['text'].str.lower()

# Create TF-IDF vectors for the training data and response data
vectorizer = TfidfVectorizer()
training_tfidf = vectorizer.fit_transform(training ['text'])
response_tfidf = vectorizer.transform(new_untrained_llm['text'])

# Compute cosine similarity between response data and training data
similarity_scores = cosine_similarity(response_tfidf, training_tfidf)
average_similarity_score = np.mean(similarity_scores)

# Print the average similarity score
print(f"Average Cosine Similarity Score:{average_similarity_score * 100:.2f}%")

Average Cosine Similarity Score:12.05%


In [ ]:
# List of test prompts and their references
test_prompts = [
    ("What is leukaemia?", "Leukaemia is a type of blood cancer."),
    ("What are the symptoms of leukaemia?", "Symptoms of leukaemia include fatigue, fever, frequent infections, and bleeding or bruising easily."),
    ("Can leukaemia be cured?", "Treatment can lead to remission, but a cure is not always possible."),
    ("Is leukaemia a type of cancer?", "Yes, leukaemia is a type of blood cancer."),
    ("Choose the correct option about leukaemia. a) leukaemia is a type of blood cancer. b) leukaemia is a kind of fruit.", "a) leukaemia is a type of blood cancer."),
    ("I had a high fever for the past 3 days, do I have leukaemia?", "High fever can be a symptom, but it does not necessarily mean you have leukaemia. Consult a doctor for proper diagnosis."),
    ("Tell me something about leukaemia.", "Leukaemia is a type of cancer that affects the blood and bone marrow.")
]

references = [ref for a, ref in test_prompts]
predictions = []

# Generate predictions
for prompt, a in test_prompts:
    response = gen_not_trained(prompt, do_sample=True, max_length=128, temperature=0.2)[0]['generated_text']
    predictions.append(response)
    print(response)

What is leukaemia?
Leukemia is a type of cancer that affects the blood cells of the body. It is a type of cancer that affects the blood cells of the body. It is a type of cancer that affects the blood cells of the body. It is a type of cancer that affects the blood cells of the body. It is a type of cancer that affects the blood cells of the body. It is a type of cancer that affects
What are the symptoms of leukaemia?
The symptoms of leukaemia are the same as those of other types of cancer. The symptoms of leukaemia are the same as those of other types of cancer. The symptoms of leukaemia are the same as those of other types of cancer. The symptoms of leukaemia are the same as those of other types of cancer.
What are the signs of leukaemia?
The signs of leukaemia are the same as those of other types of cancer. The
Can leukaemia be cured?
The answer is yes. The treatment of leukaemia is a very complex and challenging process. It is a complex process that requires a lot of research and a

In [ ]:
# Tokenize references and predictions
tokenized_references = [tokenizer.encode(ref, add_special_tokens=False) for ref in references]
tokenized_predictions = [tokenizer.encode(pred, add_special_tokens=False) for pred in predictions]

# Ensure that each list of tokens is the same length by truncating or padding
max_len = max(len(seq) for seq in tokenized_references + tokenized_predictions)
tokenized_references = [seq + [tokenizer.pad_token_id] * (max_len - len(seq)) for seq in tokenized_references]
tokenized_predictions = [seq + [tokenizer.pad_token_id] * (max_len - len(seq)) for seq in tokenized_predictions]

# Flatten the lists
flat_references = [token for seq in tokenized_references for token in seq]
flat_predictions = [token for seq in tokenized_predictions for token in seq]

# Calculate Precision, Recall, and F1 Score
precision = precision_score(flat_references, flat_predictions, average='macro', zero_division=1)
recall = recall_score(flat_references, flat_predictions, average='macro', zero_division=1)
f1 = f1_score(flat_references, flat_predictions, average='macro', zero_division=1)

print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")

Precision: 15.77%
Recall: 63.71%
F1 Score: 0.02%


In [ ]:
# Calculate BLEU scores
bleu_scores = []
for pred, ref in zip(predictions, references):
    score = sentence_bleu([ref.split()], pred.split())
    bleu_scores.append(score)

# Print average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score: {average_bleu_score * 100:.2f}%")

Average BLEU score: 2.17%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_